In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 2.0528e-02,  2.1666e-02, -7.6717e-02,  2.1393e-01,  2.0047e-01,
          6.3944e-02, -1.1214e-01,  1.5089e-01, -2.2862e-02, -4.5404e-05],
        [-6.3003e-02, -6.2372e-02, -3.1493e-02,  2.1458e-01,  2.5099e-02,
          1.9055e-01, -7.6761e-02,  1.9895e-01,  1.2513e-02, -4.6401e-03]],
       grad_fn=<AddmmBackward0>)

In [2]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[ 0.0296,  0.0919, -0.0615, -0.0125,  0.2792, -0.0454,  0.0480,  0.0243,
         -0.0038, -0.0693],
        [-0.0784,  0.0934, -0.1523, -0.1827,  0.1698, -0.0354, -0.0462, -0.1025,
          0.0090, -0.2140]], grad_fn=<AddmmBackward0>)

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 1.5235e-01, -2.9744e-02, -5.6688e-02, -6.8516e-02, -2.8941e-01,
          1.3253e-01, -2.0052e-01,  1.7656e-01, -4.1911e-02,  3.0367e-02],
        [ 3.2942e-01, -1.3150e-04, -1.1663e-01, -1.6841e-01, -1.6081e-01,
          9.8888e-02, -3.3290e-01,  1.8832e-01,  1.8696e-02, -1.4753e-01]],
       grad_fn=<AddmmBackward0>)

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [7]:
net = FixedHiddenMLP()
net(X)

tensor(-0.1108, grad_fn=<SumBackward0>)

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.3199, grad_fn=<SumBackward0>)